In [1]:
import multiprocessing
import numpy as np
import xarray as xr
import dask
from dask.distributed import Client
from pyrte_rrtmgp.data_types import RFMIPExampleFiles
from pyrte_rrtmgp.utils import load_rrtmgp_file

# try:
#     client = Client.current()
# except ValueError:
#     n_workers = multiprocessing.cpu_count()  # Get number of CPU cores available
#     client = Client(n_workers=n_workers)

from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rte_solver import rte_solve

ERROR_TOLERANCE = 1e-7

gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
atmosphere = load_rrtmgp_file(RFMIPExampleFiles.RFMIP).chunk({"expt": 3})

gas_optics_lw.compute_gas_optics(atmosphere, problem_type="absorption")

fluxes = rte_solve(atmosphere, add_to_input=False)

rlu = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RLU)
rld = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RLD)

assert np.isclose(fluxes["lw_flux_up"], rlu["rlu"], atol=ERROR_TOLERANCE).all()
assert np.isclose(fluxes["lw_flux_down"], rld["rld"], atol=ERROR_TOLERANCE).all()